Задачей даннной работы является выявления преимуществ и недостатков следующих методов оптимизации нулевого и первого порядка:
Замечание:Описание метода предсатвляет сообой переход метода от состояния a к состоинию b ввиду итеративности всех представленных методов 
1) Градиентный спуск с фиксированным шагом. Описание метода: По исходной точке a и значению градиента функции в ней вычисляется точка b как a - grad(f) * learning_rate, где learning_rate - предварительно заданное значение, неизменное на протяжении всей 
2) Градиентный спуск с изменяющимся шагом на основе метода дихотомии. Описание метода:  По исходной точке a и значению градиента функции в ней вычисляется точка b как a - grad(f) * learning_rate1, где learning_rate - результат работы метода дихотомии на функции f(learning_rate1) = a - grad(f) * learning_rate1. Левой границей метода служит 0, правой -learning_rate, задаваемый для градиентного спуска. Вычисление останавливается при достижении заданной точности 
3) Градиентный спуск с изменяющимся шагом на основе метода золотого сечения. Описание метода: аналогичен п.2, за исключением того, что вместо метода дихотомии используется метод золотого сечения
Замечание: для большей стабильности методов 1-3 градиент (в формуле вычисления следующего значения) нормируется
4) Метод покоординатного спуска. Описание метода: Основываясь на исходной точке x и длине шага step_len Итеративно проверяется точка, получаемая из x прибавлением step_len к проверяемой координате. Если значение меньше, чем в x - происходит переход в данную точку, иначе проверяется значение в точке, получаемой из x вычитанием step_len из проверяемой координаты. Если значение в ней меньше, чем в x - происходит переход, иначе аналогичные проверки проводятся по следующей координате. Если перехода не произошло после проверки всех координат, значение step_len уменьшается
5) Метод Нелдера-Мида. Описание метода:

In [ ]:
%matplotlib widget
from src.metric import CallCount, GradientCount, StepCountBeforePrecision, TrueGradientCount
from src.common.oracul import LambdaOracul, PoweredSumOracul, GradientLambdaOracul

from src.lab1.stop_condition import CountCondition, PrecisionCondition, PrecisionOrCountCondition
import numpy as np
from src.common import Oracul, Point
from src.lab1.method_processor import MethodProcessor
from src.lab1.methods import RandomMethod, GoldenRatioMethod, GradientDescent, BaseGradientDescent, DichotomyMethod, \
    CoordinateDescent

import matplotlib.pyplot as plt

fig = plt.figure()

method = RandomMethod()
golden_ratio = GoldenRatioMethod()
gradient_descent_fix = BaseGradientDescent()
gradient_descent_golden = GradientDescent()
gradient_descent_dichotomy = GradientDescent(method=DichotomyMethod())
coordinate_descent = CoordinateDescent()
oracul = LambdaOracul(lambda x: (x - 10) ** 2)
test_oracul = PoweredSumOracul([[2, 2], [0, 2]])
gradient_oracul = GradientLambdaOracul(lambda x, y: (x - 10) ** 2 + (y - 5) ** 2,
                                       lambda x, y: np.array([2 * (x - 10), 2 * (y - 5)], dtype=np.float64))
point, metrics, anim = MethodProcessor.process(gradient_descent_golden, gradient_oracul, CountCondition(20),
                                               metrics=[CallCount()],
                                               method_params={"x": np.array([100, 200]), 'learning_rate': 100},
                                               visualize=True,
                                               low_bracket=[-100, -100], high_bracket=[200, 200])
print(metrics)
print(point)
plt.show()

In [ ]:
base_test = GradientLambdaOracul(lambda x, y: x ** 2 + y ** 2,
                                 lambda x, y: np.array([2 * x, 2 * y], dtype=np.float64))
base_differential = GradientLambdaOracul(lambda x, y: 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y,
                                         lambda x, y: np.array([0.52 * x - 0.48 * y, 0.52 * y - 0.48 * x],
                                                               dtype=np.float64))
bad_defined_base = GradientLambdaOracul(lambda x, y: 100 * x ** 2 + y ** 2,
                                        lambda x, y: np.array([200 * x, 2 * y], dtype=np.float64))

In [ ]:
def tester(methods, oraculs, eps=0.001, start_point=np.array([100, 200]), visualize=True, braket_l=None, braket_h=None,
           learning_rate=300):
    res_points = []
    res_metrics = []
    part_anim_res = []
    for m in methods:
        part_point_res = []
        part_metric_res = []
        for f in oraculs:
            temp_point, temp_metrics, temp_anim = MethodProcessor.process(m, f, PrecisionCondition(eps),
                                                                          metrics=[CallCount(), GradientCount(),
                                                                                   StepCountBeforePrecision(eps)],
                                                                          method_params={"x": start_point,
                                                                                         "learning_rate": learning_rate,
                                                                                         "eps": eps},
                                                                          visualize=visualize,
                                                                          low_bracket=braket_l,
                                                                          high_bracket=braket_h)
            part_anim_res += [temp_anim]
            part_point_res += [temp_point]
            part_metric_res += [temp_metrics]
        res_points += [part_point_res]
        res_metrics += [part_metric_res]
    return res_points, res_metrics, part_anim_res


def test_fix_grad(oraculs, params, eps=0.001, start_point=np.array([100, 200]), visualize=False, braket_l=None,
                  braket_h=None):
    res_points = []
    res_metrics = []
    anim_res = []
    for f in range(len(oraculs)):
        temp_point, temp_metrics, temp_anim = MethodProcessor.process(BaseGradientDescent(), oraculs[f],
                                                                      PrecisionOrCountCondition(eps, 10 ** 5),
                                                                      metrics=[CallCount(), GradientCount(),
                                                                               StepCountBeforePrecision(eps),
                                                                               TrueGradientCount(
                                                                                   np.array([0, 0], dtype=np.float64),
                                                                                   eps)],
                                                                      method_params={"x": start_point,
                                                                                     "learning_rate": params[f],
                                                                                     "eps": eps},
                                                                      visualize=visualize,
                                                                      low_bracket=braket_l,
                                                                      high_bracket=braket_h)
        print(temp_point)
        res_points += [temp_point]
        res_metrics += [temp_metrics]
        anim_res += [temp_anim]
    return res_points, res_metrics, anim_res


In [ ]:
tester([gradient_descent_golden, gradient_descent_dichotomy, coordinate_descent],
       [base_test, base_differential, bad_defined_base], visualize=False)  #turn on visualization  to demonstration

In [ ]:
#test_fix_grad([base_test, base_differential, bad_defined_base], [0.02, 0.02001, 0.0201], visualize=False)

Замечание: Здесь и далее под методами градиентного спуска понимаются методы на основе градиентного спуска с нефиксированным шагом
Первый тест позволяет обнаружить следубщие особенности методов:
1) Как наиболее явное отличие, методы покоординатного спуска и Нелдера-Мида не требуют вычисления градиента, что является весьма значительной особенностью и выгодно выделяет их на фоне методов градиетного спуска
2) Метод градиетного спуска с фиксированным шагом, хотя и требует вычисления значений функции при данной постановке задачи, не столь сильно в ней нуждается, используя лишь для проверки приближения (как можно заметить, число шагов метода равно числу вычислений значений). Однако едва ли данное преимущество способно перекрыть такие недостатки метода, как отсутствие гарнтии сходимости и потребность в тонкой настройке параметров для обеспечения приемлимой точности или минимальной работоспособности, кроме того число вычислений градиента отличается на порядки ввиду необходимости выставления достаточно малого шага для получения приемлимой точности, кроме того возможны попадания в циклы
3) Хотя методы градиентного спуска и требуют несколько большего числа вычислений значений функции (в основном из-за необходимости вычисления шага с достаточной точностью), число их итераций для достижения искомой точности меньше, чем требуемое для покоординатного спуска *предположительно и Нелдера-Мида*
4) Метод градиентного спуска на основе дихотомии требует значительно большего числа вычисления значений функций, что является ожидаемым следствием меньшей вычислительной эффективности внутреннего метода

Точность 0.01

In [ ]:
tester([GradientDescent(method=GoldenRatioMethod(), aprox_dec=0.001),
        GradientDescent(method=DichotomyMethod(), aprox_dec=0.001), coordinate_descent],
       [base_test, base_differential, bad_defined_base], eps=0.01, visualize=False)

In [ ]:
#test_fix_grad([base_test, base_differential, bad_defined_base], [0.05, 0.05001, 0.0501], eps=0.01, visualize=False)

Точность 0.1

In [ ]:
tester([GradientDescent(method=GoldenRatioMethod(), aprox_dec=0.01),
        GradientDescent(method=DichotomyMethod(), aprox_dec=0.01), coordinate_descent],
       [base_test, base_differential, bad_defined_base], eps=0.1, visualize=False)

In [ ]:
#test_fix_grad([base_test, base_differential, bad_defined_base], [0.101, 0.1001, 0.101], eps=0.1, visualize=False)

Таким образом, при понижении точности, общие тенденции сохраняются, хотя и уменьшается разница в абсолютных значениях.
Тем не менее, стоит обратить внимание на следующие особенности:
 1) понижение точности для градиентных методов означает не только уменьшение числа вычислений функции, но и её градиента
2) Благодаря особенностям работы методов, методы градиентного спуска, как правило, вычисляют значение с некоторым запасом точности, в отличие от метода покоординатного спуска
3) Хотя запрашиваемое приближение для внутренних методов градиентых спусков (дихотомии и золотого сечения) одинаково, можно заметить, что в двух тестах число итераций градиентного спуска на основе метода золотого сечения было меньшим. Вероятно, данная особенность связана с попаданием минимума в меньший из промежутков и, как следствие, вычисление с большей реальной точностью, при меньшем объёме вычислений.
4) Использование метода градиентного спуска с фиксированным шагом на функции неизвестного вида с целью получения заданного приближения видится сомнительным
5) При уменьшении запрашиваемой точности применение метода градиентного спуска с фиксированным шагом видится более целесообразным, нежели при желании получить высокую точность, ведь хотя метод и входил в бесконечный цикл, будучи неспособным приблизиться к точке или понять ,что он уже достаточно близок, приближение происходило за разумное время

Изменённая точка старта

In [ ]:
tester([gradient_descent_golden, gradient_descent_dichotomy, coordinate_descent],
       [base_test, base_differential, bad_defined_base], start_point=np.array([10, 20]), visualize=False)

In [ ]:
tester([gradient_descent_golden, gradient_descent_dichotomy, coordinate_descent],
       [base_test, base_differential, bad_defined_base], start_point=np.array([-100, -200]), visualize=False)

Влияение выбора точки на работу метдов замечено не было.  

In [ ]:
bad_defined = GradientLambdaOracul(lambda x, y: 1000 * (np.float64(x) - 10) ** 10 + 100 * (np.float64(y) + 20) ** 10,
                                   lambda x, y: np.array(
                                       [1000 * (np.float64(x) - 10) ** 9, 1000 * (np.float64(y) + 20) ** 9],
                                       dtype=np.float64))
tester([gradient_descent_golden, gradient_descent_dichotomy, coordinate_descent],
       [bad_defined], start_point=np.array([-100, -200]), eps=0.001, visualize=True, braket_h=[100, 100],
       braket_l=[-100, -100], learning_rate=10)

In [ ]:
#test_fix_grad([bad_defined], [1], eps=0.01, visualize=True)

Таким образом, все методы справляются с выполнением своих задач на плохо обусловленных функциях(по крайней мере, до тех пор, пока их значчения представляется возможным рассчитать), хотя и наиболее эффективен здесь метод покоординатного спуска. Т.к. значение функции меняется весьма значительно даже при небольшом отклонении, методам шрадиентного спуска приходится запрашивать значительное число значений. Так же примечательно то, что метод координатного спуска сделал меньшее число итераций

In [ ]:
isom_function = GradientLambdaOracul(
    lambda x, y: -np.cos(x) * np.cos(y) * np.exp(-((x - np.pi) ** 2 + (y - np.pi) ** 2)),
    lambda x, y: np.array(
        [np.exp(-(x - np.pi) ** 2 - (y - np.pi) ** 2) * np.cos(x) * np.cos(y) + 2 * (x - np.pi) * np.cos(x),
         np.exp(-(x - np.pi) ** 2 - (y - np.pi) ** 2) * np.cos(x) * np.cos(y) + 2 * (y - np.pi) * np.cos(y)],
        dtype=np.float64))
base_mulltimodal = GradientLambdaOracul(lambda x, y: (x ** 2 + y - 11) ** 2 + (x + y ** 2 - 7) ** 2,
                                        lambda x, y: np.array([2 * (2 * x * (x ** 2 + y - 11) + x + y ** 2 - 7),
                                                               2 * (x ** 2 + 2 * y * (x + y ** 2 - 7) + y - 11)],
                                                              dtype=np.float64))


In [ ]:
tester([gradient_descent_golden, gradient_descent_dichotomy, coordinate_descent],
       [base_mulltimodal], start_point=np.array([-1000, -2000]), eps=0.0001, visualize=True, braket_h=[1000, 1000],
       braket_l=[-1000, -1000], learning_rate=10)

Ожидаемым образом, функции сходятся к оптимумам. В случае с base_multimodal - к глобальным (т.к. все являются таковыми), хотя и примечательно, что покоординатный спуск сошёлся к отличному от градиентных